# IMPORTS

In [ ]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()

In [ ]:
from AxonDeepSeg.visualization.visualize import visualize_training
import json
import os
import matplotlib.pyplot as plt
#from prettytable import PrettyTable
#from ipywidgets.widgets import *

# 1/ Build the dataset for training.

In [ ]:
# input data to build the training set
dataset_name = 'TEM_3classes_512'
gps = 0.01

In [ ]:
path_data = '../data/' + dataset_name + '/raw/'
path_testing = '../data/images_nyu_tem/'

# output path of training data path
path_training = '../data/' + dataset_name + '/training/'

In [ ]:
from scipy.misc import imread, imsave

In [ ]:
img = imread(path_testing + 'image.png')
img = 255-img
imsave(path_testing+'image-inverted.png', img)

# 2/ Load the config file and train the network

Choose here the name of the model you want to load

In [ ]:
model_name = 'cv_3c_d4_c2_k3__0-4059'

In [ ]:
# optional input path of a model to initialize the training
#path_model_init = 'network_testing/test_2905'
path_model_init = None

# output path for trained U-Net
path_model = '../models/' + model_name

path_configfile = path_model + '/config_network.json'

if not os.path.exists(path_model):
    os.makedirs(path_model)

with open(path_configfile, 'r') as fd:
    config_network = json.loads(fd.read())
    
# OPTIONAL : specify the gpu one wants to use.
gpu_device = 'gpu:0' # or gpu_device = 'gpu:1' these are the only two possible targets for now.

# TRAINING OF THE NETWORK

Uniquement nécessaire si le réseau n'a pas encore été entrainé. Sinon on va juste utiliser le modèle existant dans models

from AxonDeepSeg.train_network import train_model
train_model(path_training, path_model, config_network, path_model_init=None, gpu = gpu_device)

# 3/ Apply the model to segment one image

#### Segmentation

In [ ]:
from AxonDeepSeg.apply_model import axon_segmentation
axon_segmentation(path_testing, path_model, config_network, crop_value=25, general_pixel_size=gps)

# VISUALISATION OF THE SEGMENTATION

Import images

In [ ]:
imorg = plt.imread(path_testing + '/image.png')
imads = plt.imread(path_testing + '/AxonDeepSeg.png')

In [ ]:
plt.figure(figsize=(18,10))
plt.title('Original image')
plt.imshow(imorg, cmap='gray')

In [ ]:
plt.figure(figsize=(18,10))
plt.title('Segmented image')
plt.imshow(imads)

In [ ]:
plt.figure(figsize=(18,10))
plt.title('Superposed images')
plt.imshow(imorg, cmap='gray', alpha=0.5)
plt.imshow(imads, cmap='viridis', alpha=0.5)